In [1]:
import pandas as pd
import numpy as np
import math
import os
from test_class import Test
os.environ['PYTHONHASHSEED'] = str(50)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import matplotlib.pyplot as plt

In [2]:
rounds = 50
dataset = pd.read_csv('RTT_data.csv')
data_args = {
    'data seed': 11,
    'distance clients': [0, 2, 3, 6],
    'distance augments': [0.5, 0.5, 0.5, 0.5],
    'tolerance': 5,
    'exclude dtypes': 'object',
    'drop labels': ['GroundTruthRange[m]'],
    'target labels': ['GroundTruthRange[m]'],
    'test size 1': 0.2,
    'test size 2': 0.25
}

Mt = (np.ones(rounds) * 5).astype('int32')
model_seed = 50
test = Test(dataset, data_args, Mt, model_seed)

In [3]:
test.split(scheme = 1)

In [4]:
# perform cross validation over different seeds
seeds = [1, 2, 3, 4, 5]
fedavg_scores = []
conf_scores = []
for s in seeds:
    test.data_args['data seed'] = s
    _, _, _, _, s1, s2 = test.cross_validation(7, 1, 1000, 1, context = [0])
    fedavg_scores.append(s1[0])
    conf_scores.append(s1[1])

CV Pair 1
FedAvg
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 120.76458821159031

ConFeddi
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 167.4566386626393

FedAvg Avg Test Error: 1.8260803603996532
ConFeddi Avg Test Error: 1.9341207255979627
FedAvg MSE: 7.342414855957031
ConFeddi MSE: 7.4736223220825195

CV Pair 2
FedAvg
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 120.95919601158676

ConFeddi
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 170.339481662646

FedAvg Avg Test Error: 2.166991070225766
ConFeddi Avg Test Error: 1.1930093192438125
FedAvg MSE: 11.49260425567627
ConFeddi MSE: 3.7456295490264893

CV Pair 3
FedAvg
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 122.77307221158844

ConFeddi
Round 5
Round 10
Round 15
Round 20
Round 25
Round 3

In [5]:
np.array(fedavg_scores).mean()

1.8336635962252985

In [6]:
np.array(conf_scores).mean()

1.6331109750158022

In [7]:
np.array(fedavg_scores).var()

0.23638772520813894

In [8]:
np.array(conf_scores).var()

0.12881337510782845